### LSE Data Analytics Online Career Accelerator

# DA201: Data Analytics using Python

######################## Assignment: COVID-19 data ############################

######################## Assignment activity 1 - environment preparation ########################

In [ ]:
# Import the required libraries and set the plotting options
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

pd.options.display.max_colwidth = 200

sns.set(rc = {'figure.figsize':(15,10)})

### Required: Report submission:
Insert URL (to your public GitHub repository) and a screenshot - double click cell to edit
- [My Github Repo: Donata Pollock](https://github.com/donata-p/LSE_DA_COVID_analysis1)
- Screenshot demo (replace with your own).

!['My Github screenshot](https://github.com/donata-p/LSE_DA_COVID_analysis1/blob/main/9518FF27-3D09-40B4-818B-F13900BDDF44.jpeg?raw=true)
(Note that this only works if your repo is set to **public**. Alternatively you need to refer to a local image and include this image in your submission.)

######################## Assignment activity 2 ########################

In [ ]:
# Load the COVID-19 cases and vaccine data sets as cov and vac respectively
cases = pd.read_csv('covid_19_uk_cases.csv')
vaccinated = pd.read_csv('covid_19_uk_vaccinated.csv')
tweets = pd.read_csv('tweets.csv')

In [ ]:
# View the info()
cases.info()
vaccinated.info()
tweets.info()

In [ ]:
# View the column names
print(cases.columns)
print(vaccinated.columns)
print(tweets.columns)

In [ ]:
# Descriptive Statistics for cases
cases.describe()

In [ ]:
# Descriptive Statistics for vaccinated
vaccinated.describe()

In [ ]:
# Descriptive Statistics for twets
tweets.describe()

In [ ]:
# Data imported correctly with shape()
print(cases.shape)
cases.head()

In [ ]:
cases.tail()

In [ ]:
# Data imported correctly with shape()
print(vaccinated.shape)
vaccinated.head()

In [ ]:
vaccinated.tail()

In [ ]:
# Data imported correctly with shape()
print(tweets.shape)
tweets.head()

In [ ]:
tweets.tail()

In [ ]:
# Determine the data types
print(cases.dtypes)
print(vaccinated.dtypes)
print(tweets.dtypes)

########### Missing data

In [ ]:
# Checking cases DataFrame for missing values
cases_na = cases[cases.isna().any(axis = 1)]
print(cases_na.shape)

cases_na

In [ ]:
# Checking vaccinated DataFrame for missing values
vaccinated_na = vaccinated[vaccinated.isna().any(axis = 1)]
print(vaccinated_na.shape)

vaccinated_na 

In [ ]:
# Checking tweets DataFrame for missing values
tweets_na = tweets[tweets.isna().any(axis = 1)]
print(tweets_na.shape)

tweets_na

In cases dataframe missing values are 2.
In vaccinated dataframe missing values are 0.
Tweets missing 21 values.

########### Filter/subset data

I shall filter out 2 rows with missing values (identified in Missing Data script) in cases dataframe.

In [ ]:
# can drop missing values
# cases.dropna()

Following performing filtering on cases dataframe above left with 7582 rows × 12 columns

In [ ]:
# Filter series on Gibraltar
Gibcases = cases[cases['Province/State'] == 'Gibraltar'] 

print(Gibcases.shape)

In [ ]:
# print the Gibcases DataFrame
pd.set_option('display.max_rows', None)

Gibcases 

########### Subset the Gibraltar DataFrame that you have created consisting of the following columns: 
Deaths, Cases, Recovered and Hospitalised.

In [ ]:
# Select Deaths, Cases, Recovered and Hospitalised columns
Gibcases = pd.read_csv('covid_19_uk_cases.csv', 
                            usecols=['Deaths', 'Cases', 'Recovered', 'Hospitalised'])

In [ ]:
# Run the describe() function to generate descriptive statistics on Gibcases
Gibcases.describe()

Determining the vaccination status in Gibraltar by identifying the number of people who:
- are vaccinated
- have received the first dose
- have received the second dose.

In [ ]:
# Filter the data for the region Gibraltar on vaccinated
Gibcases_vaccinated = vaccinated[vaccinated['Province/State'] == 'Gibraltar']
Gibcases_vaccinated = Gibcases_vaccinated[['Date', 'Vaccinated', 'First Dose', 'Second Dose']]

print(Gibcases_vaccinated.shape)

In [ ]:
# Descriptive Statistics f
Gibcases_vaccinated.describe()

######################## Assignment activity 3 ########################

Identify the area that has the greatest number of people who have received a first dose but not a second dose.

In [ ]:
# Join the DataFrames as covid where you merge cov and vac
# Use the left join to merge the two DataFrames.
covid = pd.merge(cases, vaccinated,
                                    on = ['Lat', 'Long', 'Date'], how='left')
# View the new DataFrame
covid.head()

In [ ]:
# Explore the new DataFrame
print(covid.shape)
#Check for missing values
print(covid[covid.isna().any(axis = 1)].shape)

In [ ]:
# Fix the date column data type. currently it's object.
import datetime
from datetime import datetime, date
# Converting datatype of Date column to DateTime
covid['Date'] = pd.to_datetime(covid['Date'])

# Viewing dtypes
print(covid.dtypes)

In [ ]:
# drop DUPLICATED columns 
covid = covid.drop(['Province/State_y', 
                    'Country/Region_y', 
                    'ISO 3166-1 Alpha 3-Codes_y', 
                    'Sub-region Name_y', 
                    'Intermediate Region Code_y'], axis = 1)
covid.head()

In [ ]:
# clean up column names by renaming
covid = covid\
.rename(columns={'Province/State_x':'Province/State',
                 'Country/Region_x':'Country/Region',
                 'ISO 3166-1 Alpha 3-Codes_x':'ISO 3166-1 Alpha 3-Codes', 
                 'Sub-region Name_x':'Sub-region Name',
                 'Intermediate Region Code_x':'Intermediate Region Code'})

covid.head()

In [ ]:
# Groupby and calculate difference between 1st and 2nd dose by Province
difference = covid.groupby('Province/State').agg({'First Dose': 'sum', 'Second Dose': 'sum'})
difference['Outstanding Dose'] = difference['First Dose'] - difference['Second Dose']
difference

In [ ]:
# the number of cases across the UK by adding both doses together
print('Total 1st dose in GBR:', covid['First Dose'].sum())
print('Total 2nd dose in GBR:', covid['Second Dose'].sum())

Difference = covid['First Dose'].sum() - covid['Second Dose'].sum()
print('Difference between 1st and 2nd dose in GBR:', Difference)

In [ ]:
# Groupby and calculate the difference between 1st and 2nd dose over time
covid['Outstanding Dose'] = covid['First Dose'] - covid['Second Dose']
Over_Time = covid.groupby('Date').agg({'Outstanding Dose': 'sum'}).expanding(1).sum() 
Over_Time

In [ ]:
# Plot the difference between 1st and 2nd dose (Outstanding Dose) over time
ax = sns.lineplot(y = 'Outstanding Dose', x = 'Date', \
                  data = Over_Time, ci = None)
ax.set_xlabel('Date')
ax.set_title('Difference between 1st and 2nd dose over time')
plt.savefig('Outstanding Dose.png')

In [ ]:
# melt covid data
doses = pd.melt(covid,['Province/State','Date'], 
                   value_vars = ['First Dose','Second Dose','Outstanding Dose'])
doses

In [ ]:
# Plot 'Deaths','Recovered','Hospitalised'
sns.set(rc = {'figure.figsize':(15,8)})
sns.set_style('whitegrid',
                  {'grid.color':'.2','grid.linestyle':':'})

sns.lineplot(x = 'Date', y = 'value', data = doses, hue = 'variable', ci = None)
plt.title('First Dose, Second Dose, Outstanding Dose across Province/State over time')
plt.savefig('First Dose, Second Dose, Outstanding Dose across Province over time.png')

In [ ]:
Insight: peoplr who received 1st vaccine dose are gettting the 2nd dose as well

######################## Assignment activity 4 ########################

In [ ]:
# 'Difference_per_region' difference between the 1 and 2nd dose
difference['Difference_per_region'] = difference['First Dose'] - difference['Second Dose']
difference

In [ ]:
# Percentage of people who received 1st dose but not 2nd
difference['First_Percentage'] = (difference['Difference_per_region'] / difference['First Dose'])*100
difference.sort_values('First_Percentage', ascending = False).head(30)

In [ ]:
# Objective 1
# ratio of interest = Eligible*100/first dose; or (first dose - second dose)*100/first dose = first dose only.) 
difference['Ratio_of_interest'] = 100-(difference['Difference_per_region']/difference['First Dose']*100) 
difference.sort_values('Ratio_of_interest', ascending = False).head(30)

In [ ]:
# Sort and display
difference.sort_values('First_Percentage', ascending = False).head()

In [ ]:
# 1st dose to fully-dosed individuals
# reference: https://www.pythoncharts.com/python/stacked-bar-charts
plt.figure(figsize = (15,8))
sns.set_style('whitegrid',
                  {'grid.color':'.2','grid.linestyle':':'})

fullydosed_individuals = difference.plot.bar(y = ['Outstanding Dose', 'Second Dose', 'First Dose'])
plt.title('1st dose to fully-dosed individuals')
plt.ylabel('Pencentage')
plt.savefig('first_fully-dosed.png')

In [ ]:
# Objective 2
# Covid Deaths
covid_deaths = covid.groupby(['Province/State','Date']).agg({'Deaths': 'sum'})

print(covid_deaths.tail(20))
print(covid_deaths.shape)

In [ ]:
# Smooth out the data by looking at monthly figures
# excluding 'Others' Province/State that is causing the skewed data set
sns.set(rc = {'figure.figsize':(15,8)})
sns.set_style('whitegrid',
                  {'grid.color':'.2','grid.linestyle':':'})

covid['Month'] = pd.to_datetime(covid['Date']+pd.offsets.MonthBegin(0))
monthly_deaths = covid[covid['Province/State'] != 'Others'].groupby(['Month','Province/State']).agg({"Deaths": 'max'}).unstack()

monthly_deaths.plot.line()
plt.title('Monthly deaths by Province/State')
plt.ylabel('Deaths')
plt.savefig('monthly figures.png') 

In [ ]:
# Objective 3
Recovered = covid.groupby('Province/State').agg({'Recovered': 'sum'})
Recovered.sort_values('Recovered', ascending=False)

print(Recovered.head(30))
print(Recovered.shape)

In [ ]:
# excluding 'Others' Province/State that is causing the skewed data set
sns.set(rc = {'figure.figsize':(15,8)})
sns.set_style('whitegrid',
                  {'grid.color':'.2','grid.linestyle':':'})

covid['Month'] = pd.to_datetime(covid['Date'] + pd.offsets.MonthBegin(0))
Recovered = covid[covid['Province/State'] != 'Others'].groupby(['Month','Province/State']).agg({'Recovered': 'max'}).unstack()

Recovered.plot.line()
plt.title('Monthly Recovered by Province/State')
plt.ylabel('Recovered')
plt.savefig('Recovered.png') 

In [ ]:
Hospitalised = covid.groupby('Province/State').agg({'Hospitalised': 'sum'})
Hospitalised.sort_values('Hospitalised', ascending = False)

print(Hospitalised.head(30))
print(Hospitalised.shape)

In [ ]:
# excluding 'Others' Province/State that is causing the skewed data set
sns.set(rc = {'figure.figsize':(15,8)})
sns.set_style('whitegrid',
                  {'grid.color':'.2','grid.linestyle':':'})

covid['Month'] = pd.to_datetime(covid['Date'] + pd.offsets.MonthBegin(0))
Hospitalised = covid[covid['Province/State'] != 'Others'].groupby(['Month','Province/State']).agg({'Hospitalised': 'max'}).unstack()

Hospitalised.plot.line()
plt.title('Monthly Hospitalised by Province/State')
plt.ylabel('Hospitalised')
plt.savefig('Hospitalised.png') 

In [ ]:
# melt covid data
covid_summary = pd.melt(covid,['Province/State','Date'], 
                   value_vars = ['Deaths','Recovered','Cases','Hospitalised'])
covid_summary

In [ ]:
# remove cases because it skews the grapah 
covid_summary2 = covid_summary[covid_summary.variable != 'Cases']

# rename the province column so it can be filtered
covid_summary2.rename(columns = {'Province/State':'ProvinceState'}, inplace = True)

covid_summary2

In [ ]:
# Plot 'Deaths','Recovered','Hospitalised'
sns.set(rc = {'figure.figsize':(15,8)})
sns.set_style('whitegrid',
                  {'grid.color':'.2','grid.linestyle':':'})

sns.lineplot(x = 'Date', y = 'value', data = covid_summary2, hue = 'variable', ci = None)
plt.title('Deaths, Recovered, Hospitalised across ALL Province/State')
plt.savefig('Deaths, Recovered, Hospitalised across ALL Province.png')

In [ ]:
# Plot Cases separately
sns.set(rc = {'figure.figsize':(15,8)})
sns.set_style('whitegrid',
                  {'grid.color':'.2','grid.linestyle':':'})

sns.lineplot(x = 'Date', y = 'Cases', data = covid) \
.set_title('Cases across ALL Province/State')
plt.savefig('cases.png') 

In [ ]:
# Plot doesn't work because cases skew the grapah
fig, ax = plt.subplots()
ax = sns.lineplot(x ='Date', y ='Deaths', data = covid)
ax1 = sns.lineplot(x ='Date', y ='Recovered', data = covid)
ax2 = sns.lineplot(x ='Date', y ='Cases', data =covid)
ax3 = sns.lineplot(x ='Date', y ='Hospitalised', data = covid)

plt.xlabel('Date', fontsize = 15);
plt.ylabel('count', fontsize = 15);
plt.title('Deaths,Recovered,Cases,Hospitalised', fontsize = 20)
plt.tick_params(axis ='both', which ='major', labelsize = 10)
fig.savefig('Deaths, Recovered, Hospitalised across ALL Province2.png')

######################## Assignment activity 5 ########################

In [ ]:
# Imported the tweet data set in 2.1 tweets = pd.read_csv('tweets.csv')
tweets.info()

In [ ]:
# Explored the data: info(), head() in 2.1
tweets.head()

In [ ]:
# View the output
print(tweets.shape)

In [ ]:
# Explore the structure, count the tweets, get the elements of interest
tweets.retweet_count.value_counts()

In [ ]:
tweets.favorite_count.value_counts()

In [ ]:
# Create a DataFrame with the text only
# convert the text of the tweets to string datatype
tweets['text'] = tweets['text'].astype(str)

# using a lambda function, allocate the looped text only to a new DataFrame
tweets_text = tweets['text'].apply(lambda x: x if x.strip() != None else None)
tweets_text

In [ ]:
print(tweets_text.shape)

In [ ]:
# a list of values containing #
hashtags = []
for y in [x.split(' ') for x in tweets_text.values]:
    for z in y:
        if '#' in z:
            hashtags.append(z)
hashtags = pd.Series(hashtags).value_counts()

In [ ]:
# Filter and sort
hashtags.head(30)

In [ ]:
# create a new DataFrame to store the output of tags
data = pd.DataFrame(hashtags).reset_index()

# label the columns
data.columns = ['word', 'word count']

# change the count datatype to integer
data['word count'] = data['word count'].astype(int)

In [ ]:
# print the dataframe with top 30 record
data.head(30)

In [ ]:
# Plot
plt.figure(figsize=(15,8))
plt.title('Most occuring hashtags')
ax = sns.barplot(x = 'word count', y = 'word', data = data.loc[(data['word count'] > 50)])
plt.savefig('Hastags.png')

In [ ]:
# hashtags with words COVID, corona and vaccinated
COVID_corona = ['covid', 'corona','vaccinated']
covid_hashtags = data[[bool([1 for search in COVID_corona if search in x]) for x in hashtags.index]]
covid_hashtags.head(20)

In [ ]:
# Plot
plt.figure(figsize = (15,8))
plt.title('Most occuring COVID, corona, vaccinated hashtags')
ax = sns.barplot(x = 'word count', y = 'word', data = covid_hashtags.loc[(covid_hashtags['word count'] > 3)])
plt.savefig('COVID_corona_Hastags.png')

######################## Assignment activity 6 ########################

In [ ]:
# You can copy and paste the relevant code cells from the provided template here.
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load and select relevant subset of the data
# Make sure to change the relative path to function in your environment
ds1 = pd.read_csv('covid_19_uk_cases.csv')
ds2 = pd.read_csv('covid_19_uk_vaccinated.csv')

sample = ds1[['Province/State','Date','Hospitalised']]

In [ ]:
# Select data for a specific province
sample_ci = sample[sample['Province/State'] == 'Channel Islands']
# == 'Channel Islands'
# == 'Anguilla'                                     
# == 'Bermuda'                                     
# == 'British Virgin Islands'                       
# == 'Cayman Islands'                              
# == 'Channel Islands'                              
# == 'Falkland Islands (Malvinas)''                 
# == 'Gibraltar'                                     
# == 'Isle of Man'                                  
# == 'Montserrat'                                     
# == 'Others'                                          
# == 'Saint Helena, Ascension and Tristan da Cunha'   
# == 'Turks and Caicos Islands'                      

In [ ]:
# Exploring and viewing data

print(sample_ci.shape)
print(sample_ci.columns)
print(sample_ci.dtypes)

sample_ci.head()

In [ ]:
# Plot 
sample_ci.plot(figsize = (12, 4))

# Specify the legend and title of the plot.
plt.legend(loc = 'best')
plt.title('Hospitalisations in Channel Islands')
plt.show(block = False);
plt.savefig('Hospitalisations in Channel Islands.png')
# Check for missing values.
sample_ci.isna().sum()

In [ ]:
# Define function to plot moving averages
def plot_moving_average(series, window, plot_intervals=False, scale = 1.96):
    
    # Create a rolling window to calculate the rolling mean using the series.rolling function
    rolling_mean = series.rolling(window = window).mean()
    
    # Declare the dimensions for the plot, plot name and plot the data consisting of the rolling mean from above 
    plt.figure(figsize=(18,4))
    plt.title('Moving average\n window size = {}'.format(window))
    plt.plot(rolling_mean, 'g', label='Simple moving average trend')

    
    # Plot confidence intervals for smoothed values
    if plot_intervals:
        
        # Calculate the mean absolute square 
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        
        # Calculate the standard deviation using numpy's std function
        deviation = np.std(series[window:] - rolling_mean[window:])
        
        # Calculate the upper and lower bounds 
        lower_bound = rolling_mean - (mae + scale * deviation)
        upper_bound = rolling_mean + (mae + scale * deviation)
        
        # Name and style upper and lower bound lines and labels 
        plt.plot(upper_bound, 'r--', label = 'Upper bound / Lower bound')
        plt.plot(lower_bound, 'r--')
    
    # Plot the actual values for the entire timeframe
    plt.plot(series[window:], label = 'Actual values')
    plt.grid(True)

In [ ]:
# Define function to calculate the mean absolute error
def mean_absolute_error(a, b): return abs(b - a)

In [ ]:
plot_moving_average(sample_ci.Hospitalised, 7, plot_intervals = True)

In [ ]:
# 7 days:
plot_moving_average(sample_ci.Hospitalised, 7, plot_intervals = True)

# 14-days smoothing:
plot_moving_average(sample_ci.Hospitalised, 14, plot_intervals = True)

# 21-days smoothing:
plot_moving_average(sample_ci.Hospitalised, 21, plot_intervals = True)

The user defined function takes a series, a window frame, plot_intervals and scale as arugments. It is used to calculate and plot 'Simple moving average' for the column in the dataframe for the window of given number of entries in the date column, in this case days, which are passed as arguments to the functions.
The code written above by me calls the function to calculate and plot the 'Simple moving average' for the 'Hospitalised' column in the data frame with 'Channel Islands' data for a window of 7 days. It also establishes the confidence intervals when plot_intervals is set to true.

In [ ]:
# Return the top three days with biggest difference between daily value and rolling 7-day mean
s = sample_ci.copy()
s_rolling = s['Hospitalised'].rolling(window=7).mean()
s['error'] = mean_absolute_error(s['Hospitalised'][7:], s_rolling[7:])
s.sort_values('error', ascending = False).head(3)

In [ ]:
# Code explanation
# Return the top three days with biggest difference between daily value and rolling 7-day mean

s = sample_ci.copy() # create dataset 

s_rolling = s['Hospitalised'].rolling(window = 7).mean() # create series of rolling mean

s['error'] = mean_absolute_error(s['Hospitalised'][7:], s_rolling[7:]) # calculate absolute error for each row,omitting first 7 

s.sort_values('error', ascending = False).head(3) # sort. print

Mean_absolute_error (MAE) calculates difference between daily value and rolling 7 day mean (moving window function of Pandas) on 'Hospitalised' column. Values are assigned to 'error' column are caculated by subtracting actual value ('Hospitalized' column) by calculated value (s_rolling DataFrame). Then sort_values in descending order. Head returns 3 values.

In [ ]:
print(s.loc[[2593]])
print(s_rolling.loc[[2593]])

######################## Further exploration ########################

In [ ]:
fig = plt.figure(figsize = (18, 4))

ax = sns.boxplot(x = sample_ci['Hospitalised'], whis = 1.5)

In [ ]:
# The columns you want to search for outliers in.
cols = ['Hospitalised'] 

# Calculate quantiles and IQR.
# Same as np.percentile but maps (0,1) and not (0,100).
Q1 = sample_ci[cols].quantile(0.25) 
Q3 = sample_ci[cols].quantile(0.75)
IQR = Q3 - Q1
IQR

# Return a Boolean array of the rows with (any) non-outlier column values.
condition = ~((sample_ci[cols] < (Q1 - 1.5 * IQR)) | (sample_ci[cols] > (Q3 + 1.5 * IQR))).any(axis=1)

# Filter our DataFrame based on condition.
sample_ci_non_outlier = sample_ci[condition]
sample_ci_non_outlier.shape

In [ ]:
# Plot to see if outliers have been removed: 
# whis=multiplicative factor.
fig = plt.subplots(figsize = (18, 4))

ax = sns.boxplot(x = sample_ci_non_outlier['Hospitalised'],whis = 1.5)

Week 6 Questions:

Question 1: We have heard of both qualitative and quantitative data from the previous consultant. What are the differences between the two? Should we use only one or both of these types of data and why? How can these be used in business predictions? Could you provide examples of each?

Answer 1: Quantitative data refers to any information which can be quantified, measured or counted. Examples in our project include vaccination rates, calculated percentages. Qualitative data is more descriptive and based on language e.g. Twitter posts.

Question 2: We have also heard a bit about the need for continuous improvement. Why should this be implemented, it seems like a waste of time. Why can’t we just implement the current project as it stands and move on to other pressing matters?

Answer 2: We should always be looking to continally improve our analysis to understand causes.

Question 3: As a government, we adhere to all data protection requirements and have good governance in place. We only work with aggregated data and therefore will not expose any personal details. Have we covered everything from a data ethics standpoint? Is there anything else we need to implement from a data ethics perspective?

Answer 3: Whilst we are using aggregated data to perform this analysis, the initial data collection would still have some degree of personal infomation.